In [1]:
# convert n2c_fp results to standard format

In [19]:
import numpy as np
import glob
import os
import sys

In [32]:
dose = 0.125

# get file list, this is the patient order of the testing slices
patients = [os.path.basename(s).split('_')[0] for s in
            glob.glob('/home/dwu/DeepRecon/train/data/real_2D_3_layer_mean/*_full_hann.npy')]

# get folders of testing results
testingFolders = glob.glob('../../train/recon/n2c_fp/encoder_decoder/%g/L*'%dose)

# load reference, the saved rmse was wrong
refs = np.load('../../train/recon/data/fp/full_hann.npy') / 0.019

outDir = '../../train/recon/n2c_fp/encoder_decoder/%g/final'%dose
nSlicesPerPatient = 10

In [33]:
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [34]:
for folder in testingFolders:
    tokens = os.path.basename(folder).split('_')
    imgs = np.load(os.path.join(folder, 'recon.npy'))
    
    baseSlices = [patients.index(tokens[0]) * 10, patients.index(tokens[1]) * 10]
    imgs = [imgs[:nSlicesPerPatient, ...], imgs[nSlicesPerPatient:, ...]]
    
    
    for img, baseSlice in zip(imgs, baseSlices):
        for iSlice in range(nSlicesPerPatient):
            rmse = np.sqrt(np.mean((img[iSlice, 128:-128, 128:-128, 0] - 
                                    refs[iSlice + baseSlice, 128:-128, 128:-128, 0])**2))
            
            curDir = os.path.join(outDir, str(iSlice + baseSlice))
            if not os.path.exists(curDir):
                os.makedirs(curDir)
            np.save(os.path.join(curDir, 'recon.npy'), img[iSlice, ...])
            np.savez(os.path.join(curDir, 'loss.npz'), header='rmseRoi', val=rmse)

In [25]:
rmse

0.9473883